In [1]:
import pandas as pd
import numpy as np
from pmdarima.arima import AutoARIMA
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from datetime import date as dt, timedelta
import yfinance as yf
import pymysql
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader
import datetime as dt
import pandas_datareader.data as web
from dateutil.relativedelta import *

In [2]:
conn=pymysql.connect(host='*******',port=int(*****),user='*******',passwd='********',db='Options')
df_all =pd.read_sql_query('SELECT * FROM options.unusual_stock_options_activity;',conn)

In [ ]:
#Seperating into call df and put df. Also making the dates datetime objects 

def put_call_df(df):
    '''
    input = df 
    output = df_call, df_put --> two df with it's call and put version   
    '''
    df_call = df[df.Type == 'Call']
    df_put = df[df.Type =='Put']
    df_call.Exp_Date = pd.to_datetime(df_call['Exp_Date']).astype('datetime64[D]')
    df_put.Exp_Date = pd.to_datetime(df_put['Exp_Date']).astype('datetime64[D]')
    df_call.date_traded = pd.to_datetime(df_call['date_traded']).astype('datetime64[D]')
    df_put.date_traded = pd.to_datetime(df_put['date_traded']).astype('datetime64[D]')
    
    return df_call, df_put

In [ ]:
# df_call, df_put = put_call_df(df_all)
# df_put

In [ ]:
# Calculating the expiration Volume for both df_call and df_put that was segregated above

def total_exp_volume(df_call, df_put):
    
    '''
    input = df_call, df_put --> for a particular stock
    output = df_call, df_put with it's calculated expiration volume for a expiration date    
    '''
    
    call_exp_vol = []
    put_exp_vol = []
    for item in df_call.Exp_Date:
        vols = df_call[df_call.Exp_Date == item].Volume.sum()
        call_exp_vol.append(vols)
    for item in df_put.Exp_Date:
        vols = df_put[df_put.Exp_Date == item].Volume.sum()
        put_exp_vol.append(vols)

    df_call["total_exp_vol"] = call_exp_vol
    df_put['total_exp_vol'] = put_exp_vol
    
    CallPutDf = pd.DataFrame(columns=["Date1", "CallPut_exp"])
    date_list = []
    value_list = []
    
    call_list = df_call.Exp_Date.unique().astype('datetime64[D]')
    put_list = df_put.Exp_Date.unique().astype('datetime64[D]')
    call_list = list(set(call_list) | set(put_list))
    
    
    for date in call_list:
        if len(df_call[df_call.Exp_Date == date])>0 and len(df_put[df_put.Exp_Date == date])>0 :
            total_day_vol = df_call[df_call.Exp_Date == date].total_exp_vol.iloc[0]/df_put[df_put.Exp_Date == date].total_exp_vol.iloc[0]
            date_list.append(date)
            value_list.append(total_day_vol)

        elif len(df_call[df_call.Exp_Date == date])>0 and len(df_put[df_put.Exp_Date == date])==0:
            total_day_volm = df_call[df_call.Exp_Date == date].total_exp_vol.iloc[0]
            date_list.append(date)
            value_list.append(total_day_volm)

        elif len(df_call[df_call.Exp_Date == date])==0 and len(df_put[df_put.Exp_Date == date])>0:
            total_day_volmn = df_put[df_put.Exp_Date == date].total_exp_vol.iloc[0]
            date_list.append(date)
            value_list.append(total_day_volmn)
            
    CallPutDf['Exp_Date'] = date_list
    CallPutDf["CallPut_exp"] = value_list
#     CallPutDf["CallPut_exp"] = round(CallPutDf["CallPut_day"],2)
    
    df_call = df_call.merge(CallPutDf,how='left', left_on="Exp_Date", right_on='Exp_Date')
    df_put = df_put.merge(CallPutDf,how='left', left_on="Exp_Date", right_on='Exp_Date' )
            
            
    return df_call, df_put

In [ ]:
# df_call, df_put = total_exp_volume(df_call, df_put)

In [ ]:
# resultList = list(set(call_date) | set(put_date))

In [ ]:
#Calculating the day volume and Calls/Puts Volume ratio for that particular date 
    
def total_day_volume(df_call, df_put):
    
    '''
    input = df_call, df_put --> for a particular stock
    output = df_call, df_put with it's calculated daily    
    '''  
    
    call_day_vol = []
    put_day_vol = []
    for item in df_call.date_traded:
        vols = df_call[df_call.date_traded == item].Volume.sum()
        call_day_vol.append(vols)
    for item in df_put.date_traded:
        vols = df_put[df_put.date_traded == item].Volume.sum()
        put_day_vol.append(vols)

    df_call["total_day_vol"] = call_day_vol
    df_put['total_day_vol'] = put_day_vol
    
    #The code from below there can be commented out in for debugging purposes, below code works fine individually
    
    
    CallPutDf = pd.DataFrame(columns=["Date", "CallPut_day"])
    date_list = []
    value_list = []
    
    call_list = df_call.date_traded.unique().astype('datetime64[D]')
    put_list = df_put.date_traded.unique().astype('datetime64[D]')
    call_list = list(set(call_list) | set(put_list))
#     for items in df_put.date_traded:
#         if items not in call_list:
#             call_list.append(items)

    for date in call_list:
        if len(df_call[df_call.date_traded == date])>0 and len(df_put[df_put.date_traded == date])>0 :
            total_day_vol = df_call[df_call.date_traded == date].total_day_vol.iloc[0]/df_put[df_put.date_traded == date].total_day_vol.iloc[0]
            date_list.append(date)
            value_list.append(total_day_vol)

        elif len(df_call[df_call.date_traded == date])>0 and len(df_put[df_put.date_traded == date])==0:
            total_day_volm = df_call[df_call.date_traded == date].total_day_vol.iloc[0]
            date_list.append(date)
            value_list.append(total_day_volm)

        elif len(df_call[df_call.date_traded == date])==0 and len(df_put[df_put.date_traded == date])>0:
            total_day_volmn = df_put[df_put.date_traded == date].total_day_vol.iloc[0]
            date_list.append(date)
            value_list.append(total_day_volmn)
            
    CallPutDf['date_traded'] = date_list
    CallPutDf["CallPut_day"] = value_list
#     CallPutDf["CallPut_day"] = round(CallPutDf["CallPut_day"],2)
    
    df_call = df_call.merge(CallPutDf,how='left', left_on="date_traded", right_on='date_traded')
    df_put = df_put.merge(CallPutDf,how='left', left_on="date_traded", right_on='date_traded' )
    
    return df_call, df_put

In [ ]:
def final_func(df_all):

    start = datetime.datetime.now()
    comb_list = df_all.Symbol.unique()
    new_df = pd.DataFrame(columns=df_all.columns)
    new_df["total_exp_vol"] = ""
    new_df["total_day_vol"] = ""

    # comb_list = comb_list[:1]

    for comb in comb_list:
        df = df_all[df_all.Symbol==comb]
        df_call, df_put = put_call_df(df)
        df_call, df_put = total_exp_volume(df_call, df_put)
        df_call, df_put = total_day_volume(df_call, df_put)

        new_df = new_df.append(df_call, ignore_index=True)
        new_df = new_df.append(df_put, ignore_index=True)


    new_df["total_exp_vol"] = new_df["total_exp_vol"].astype("int")
    new_df["total_day_vol"] = new_df["total_day_vol"].astype("int")
    new_df["Exp_Date"] = pd.to_datetime(new_df['Exp_Date']).dt.date
    new_df["date_traded"] = pd.to_datetime(new_df['date_traded']).dt.date
    new_df.drop(columns="Date", inplace=True)
    new_df.drop(columns="Date1", inplace=True)

    print(f"Total time for execution = {datetime.datetime.now() - start}")

    return new_df

In [ ]:
new_df = final_func(df_all)
new_df
# new_df.to_excel("test_df.xlsx")